# Query a pre-built database of chess positions

This demo shows you how to search a pre-bulit database of chess positions for positions that are similar to your own query position.

### Enter your query

First you need to specify the position for which you want to find similar positions for. Use the widget in the following cell to set up a board and then press the *Export Position as White* button if it is White's turn and *Export Position as Black* button if it is Blacks's turn.

In [ ]:
%%html
<style>
.spare-pieces-7492f img {
    display: inline-block
}
</style>

<link rel="stylesheet" href="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.css" integrity="sha384-q94+BZtLrkL1/ohfjR8c6L+A6qzNH9R2hBLwyoAfu3i/WCvQjzL2RQJ3uNHDISdU" crossorigin="anonymous">

<div id="myBoard" style="width: 400px"></div>
<button id="startBtn">Start Position</button>
<button id="clearBtn">Clear Board</button>
<button id="exportWhite">Export Position as White</button>
<button id="exportBlack">Export Position as Black</button>

<script src="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.js" integrity="sha384-8Vi8VHwn3vjQ9eUHUxex3JSN/NFqUg3QbPyX8kWyb93+8AC/pPWTzj+nHtbC5bxD" crossorigin="anonymous"></script>

<script>
var board = Chessboard('myBoard', {
  draggable: true,
  dropOffBoard: 'trash',
  sparePieces: true
});
var fen = undefined;
document.getElementById('startBtn').onclick = board.start;
document.getElementById('clearBtn').onclick = board.clear;
document.getElementById('exportWhite').onclick = function(){
    var fen = board.fen();
    IPython.notebook.kernel.execute("fen='"+fen+" w'");
};
document.getElementById('exportBlack').onclick = function(){
    var fen = board.fen();
    IPython.notebook.kernel.execute("fen='"+fen+" b'");
};
</script>

In [ ]:
print(f"The fen string of your position is: {fen}")

**Alternatively** if the widget above does not work you can visit [lichess.org](https://lichess.org/editor) retrieve the fen string of your position there and then paste it in the cell below.

In [11]:
'''
Execute this cell if you don't want to use the widget above.
Replace the example fen string with your own.
'''
fen = "rnbq1rk1/pp2bppp/2p1pn2/4N1B1/2pP4/2N3P1/PP2PPBP/R2Q1RK1 b Qq - 0 1"

### Load the database of chess positions

In [5]:
import faiss
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from searchpos import *

Load the database into memory. *You need more than 1.6GB of RAM.*

In [6]:
filepath = "../data/test.faiss"
index = index_load(filepath, is_binary=True)

Prepare the search query

In [32]:
search_results = 10
query = [fen,"r1k4r/p2nb1p1/2b4p/1p1n1p2/2PP4/3Q1NB1/1P3PPP/R5K1 w - - 0 1"]

Search database and retrieve most similar positions

In [33]:
dist, reconstructed = index_query_positions(query, index, input_format='fen', output_format='board',
                                            num_results=search_results)


 (2, 97)


In [34]:
print(dist)

[[ 4  6  6  7  7  7  7  7  7  7]
 [14 14 15 15 15 15 15 15 16 16]]


In [36]:
from IPython.display import HTML
html = '''
<link rel="stylesheet" href="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.css" integrity="sha384-q94+BZtLrkL1/ohfjR8c6L+A6qzNH9R2hBLwyoAfu3i/WCvQjzL2RQJ3uNHDISdU" crossorigin="anonymous">
<table>
'''
for i in range(len(reconstructed)):
    html += f'''
  <tr>
    <td>Your Query Position {i}</td>
    <td>
      <select id="mySelect{i}" onchange="myFunction{i}()">
'''
    for j in range(search_results):
        html += f'''<option value='{reconstructed[0][j].fen()}'>Similar Position {j}</option>'''
    html += f'''
      </select>
    </td>
  </tr>
  <tr>
    <td><div id="query{i}" style="width: 400px"></div></td>
    <td><div id="myBoard{i}" style="width: 400px"></div></td>
  </tr>
'''
html += '''
</table>

<script src="https://unpkg.com/@chrisoakman/chessboardjs@1.0.0/dist/chessboard-1.0.0.min.js" integrity="sha384-8Vi8VHwn3vjQ9eUHUxex3JSN/NFqUg3QbPyX8kWyb93+8AC/pPWTzj+nHtbC5bxD" crossorigin="anonymous"></script>

<script>
'''
for i in range(len(reconstructed)):
    html += f'''
var pos{i} = document.getElementById("mySelect{i}").value;
var board{i} = Chessboard('myBoard{i}',{{showNotation: false}});
var query{i} = Chessboard('query{i}',{{
  position: '{query[i]}',
  showNotation: false
}});

function myFunction{i}() {{
  var position = document.getElementById("mySelect{i}").value;
  board{i}.position(position);
}}
'''
html += '''
</script>
'''
HTML(html)

Your Query Position 0,Similar Position 0Similar Position 1Similar Position 2Similar Position 3Similar Position 4Similar Position 5Similar Position 6Similar Position 7Similar Position 8Similar Position 9
,
Your Query Position 1,Similar Position 0Similar Position 1Similar Position 2Similar Position 3Similar Position 4Similar Position 5Similar Position 6Similar Position 7Similar Position 8Similar Position 9
,
